# Introduction

This notebook will walk you through creating and monitoring HITs, specifically for the importance labeling task.

It provides methods to create HITs, pretty-print HIT and assignment status, expire/edit HITs, create qualifications, and download collected data. 

Before continuing, make sure that you have read the README and set all config fields to their desired values.

## Requirements: 

This code requires Python3 and the following packages: 
- boto3 
- beautiful soup 4

Before using, you will have to set up an authentication key to use the Amazon API and include it in a credentials file. See here: https://aws.amazon.com/developers/getting-started/python/

# Setup

Read the config file and establish a connection to MTurk.

A connection is made to production or to the sandbox based on values in the config. 

In [1]:
import datetime
import boto3
import json
import copy
import pprint
import os
from bs4 import BeautifulSoup as bs 
from uuid import uuid4

In [2]:
# Constants
CONFIG_PATH=None

# Path to the html task
TASK_PATH = '..'
title = "Detect deepfakes appearing for fractions of a second!"
description = "You will be shown short videos of faces, sometimes for fractions of a second. Some of them are deepfakes. Your task is to determine whether the videos shown are real or deepfakes."
task_url = "https://cfosco.github.io/timed_df_detection_experiment/"


# if TASK_PATH == '../../df_click':
#     title = "Annotate Artifacts in Deepfakes!"
#     description = 'You will be shown deepfake videos. Your task is to annotate the artifacts and defects of the fake video: all the areas on the face that do not look real.'
#     task_url = "https://allenjlee.github.io/dfclick/"
# elif TASK_PATH == '../../df_exp':
#     title = "Detect deepfakes without moving your eyes"
#     description = "you will be shown to video side by side. One of them is a fake, without moving your eyes from a fixation cross in the middle of the videos, you have to detect which one is fake."
#     task_url = "https://allenjlee.github.io/df_exp/?data="
# else:
#     print("Task path not recognized")

# where to save downloaded results 
SAVE_PATH = './results'

# sandbox or prod
USING_PROD = True

In [3]:
# Safety flags that prevent you from accidentally messing up your HITs. 
# Set to False except when you are performing these specific tasks.
ALLOW_HIT_CREATION = True
ALLOW_ASSIGNMENT_ADDITION = False
ALLOW_CREATE_QUAL = True
ALLOW_UPDATE_EXPIRATION = True

In [4]:
# Read config and extract relevant settings

if CONFIG_PATH:
    with open(CONFIG_PATH, 'r') as f:
        config = json.loads(f.read())
else:
    config = {"hitCreation": {
            "title": title,
            "description": description,
            "numTasks": 1,
            "numAssignments": 10,
            "rewardAmount": "1.0",
            "keywords": "deepfakes, videos, artifacts, detection, annotate",
            "duration": 2000,
            "lifetime": 186400,
            "taskUrl": task_url,
            "production": USING_PROD # Change between sandbox and prod here
        },

        "advanced": {
            "includeDemographicSurvey": False,
            "hideIfNotAccepted": False,
            "externalSubmit": False,
            "externalSubmitUrl": ""
        }}

    
hit_config = config['hitCreation']

external_submit = config['advanced']['externalSubmit']

if USING_PROD:
    print("USING PROD")
    endpoint_url = 'https://mturk-requester.us-east-1.amazonaws.com'
    origin="production"
else:
    print("USING SANDBOX")
    endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
    origin="sandbox"

# If using an external link, add a querystring origin=sandbox or origin=production 
# for use in your js logic if you want. Not done for MTurk submits because it breaks the submit link
if external_submit: 
    hit_config['taskUrl'] = "%s?origin=%s" % (hit_config['taskUrl'], origin)

if external_submit:
    print("Configuring task as external link with data submitted to: %s" % config['advanced']['externalSubmitUrl'])
else:
    print("Configuring task as an iframe within Mturk")

session = boto3.session.Session(profile_name="default")
cl = session.client('mturk', region_name='us-east-1', endpoint_url=endpoint_url)

USING PROD
Configuring task as an iframe within Mturk


## Define task URL with folds to use in HIT
The folds are txt files containing paths to the images to be used in the hit. This notebooks allows for creation of multiple hits, each with a different fold.txt .
In the folder "files", multiple txt files (folds) can be found. Each one contains 10 images, and there are no repeats among folds. Each fold has a piece of the full canva_scraping2 dataset.

In [5]:
# Whether to launch a hit per fold.txt in folder "files"
LAUNCH_HITS_FOR_MULTIPLE_FOLDS = True
FOLD_REQ_STR = 'json_'
fold_numbers = list(range(32,50))

# # # If need to manually specify fold, can be done here
# fld = "test.json" #'sentinel_large.txt'

if LAUNCH_HITS_FOR_MULTIPLE_FOLDS:
    if fold_numbers is not None:
        all_folds = [f for f in os.listdir(os.path.join(TASK_PATH,'src/jsons_2vig')) if (FOLD_REQ_STR in f and int(f.split('.')[0].split('_')[-1]) in fold_numbers)]    
    else:    
        all_folds = [f for f in os.listdir(os.path.join(TASK_PATH,'src/jsons_2vig')) if (FOLD_REQ_STR in f)]    

    print('Using folds:',all_folds)
    print('Num folds to use:', len(all_folds))
else:
    if fld:
        hit_config['taskUrl'] = hit_config['taskUrl'] + "?data=%s" % fld
    elif hit_config['fold']:
        hit_config['taskUrl'] = hit_config['taskUrl'] + "?data=%s" % hit_config['fold']
    else:
        fld = input('Define fold (input name+.txt)')
        hit_config['taskUrl'] = hit_config['taskUrl'] + "?data=%s" % fld    
    
print("TASK URL:", hit_config['taskUrl'])

Using folds: ['json_38.json', 'json_39.json', 'json_40.json', 'json_43.json', 'json_36.json', 'json_44.json', 'json_37.json', 'json_34.json', 'json_41.json', 'json_46.json', 'json_33.json', 'json_42.json', 'json_45.json', 'json_47.json', 'json_35.json', 'json_32.json']
Num folds to use: 16
TASK URL: https://cfosco.github.io/timed_df_detection_experiment/


# Make new HIT

In [65]:
# List of qualifications that you will use to filter potential workers. 
# These require that workers come from the US and have an approval rating >= 99%
# Edit this list to specify different qualifications for workers 
QUALS = [
    {
        'QualificationTypeId': '00000000000000000071',
        'Comparator': 'EqualTo',
        'LocaleValues': [{
            'Country': 'US',
        }],
    },
    {
        'QualificationTypeId': '000000000000000000L0',
        'Comparator': 'GreaterThanOrEqualTo',
        'IntegerValues': [
            98
        ],
    },

]

if USING_PROD:
    QUALS.append(
            {  # Adding qualification to blacklist workers
                'QualificationTypeId': '3CNIZ8EIUXRXLUMEOJNMO8EPWSI3BW', 
                'Comparator': 'DoesNotExist',
             }
    )

In [66]:
# Helpers for creating HITs. 

# generic helper that sets metadata fields based on the config file.
def create_hit(task, questionText, quals=QUALS): 
    response = cl.create_hit(
        MaxAssignments=task['numAssignments'],
        AutoApprovalDelayInSeconds=604800,
        LifetimeInSeconds=task['lifetime'],
        AssignmentDurationInSeconds=task['duration'],
        Reward=task['rewardAmount'],
        Title=task['title'],
        Keywords=task['keywords'],
        Description=task['description'],
        Question=questionText,
        QualificationRequirements=quals,
    )
    print(response)
    print("\n")

# creates a HIT in the form of an External Question inside an iFrame
def create_hit_iframe(task):
    questionText = "<ExternalQuestion xmlns=\"http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/"
    questionText += "2006-07-14/ExternalQuestion.xsd\">\n<ExternalURL>" + task['taskUrl']
    questionText += "</ExternalURL>\n  <FrameHeight>700</FrameHeight>\n</ExternalQuestion>"
    create_hit(task, questionText)
    
# Helper to create a HIT in the form of a simple UI with a link to an external page and an
# input box for a completion code 
def create_hit_external(task):
    with open('questionform_template.xml', 'r') as myfile:
        template=myfile.read() 
    question_xml = template % (hit_config["title"], hit_config["description"], task['taskUrl'])
    create_hit(task, question_xml)

In [67]:
# Use this cell to launch your HIT! 
if ALLOW_HIT_CREATION: 
    if not (hit_config.get('variants', False) or hit_config.get('numTasks', False)): 
        raise RuntimeError("You must specify either hitCreation.numTasks or hitCreation.variants in your config.json file")
    
    hit_creation_function = create_hit_external if external_submit else create_hit_iframe
    
    if LAUNCH_HITS_FOR_MULTIPLE_FOLDS:
        print("creating", len(all_folds), "tasks with folds %s" % all_folds)
        for fold in all_folds:
            hit_config['taskUrl'] = hit_config['taskUrl'].split('?')[0] + "?data=%s" % fold
            print("Creating HIT with %d assignments and url %s" % (hit_config['numAssignments'], hit_config['taskUrl']))
            hit_creation_function(hit_config)
    
    elif hit_config.get('numTasks', False): 
        print("creating " + str(hit_config['numTasks']) + " tasks")
        for i in range(hit_config['numTasks']):
            hit_creation_function(hit_config)
    else: 
        print("creating " + str(len(config['variants'])) + " variants")
        for var in hit_config['variants']: 
            task = copy.deepcopy(config)
            task.update(var)
            hit_creation_function(task)
    
else: 
    raise RuntimeError("This action is not currently enabled; set `ALLOW_HIT_CREATION` to true to proceed with this action")

creating 16 tasks with folds ['json_38.json', 'json_39.json', 'json_40.json', 'json_43.json', 'json_36.json', 'json_44.json', 'json_37.json', 'json_34.json', 'json_41.json', 'json_46.json', 'json_33.json', 'json_42.json', 'json_45.json', 'json_47.json', 'json_35.json', 'json_32.json']
Creating HIT with 10 assignments and url https://cfosco.github.io/timed_df_detection_experiment/?data=json_38.json
{'HIT': {'HITId': '374UMBUHN74T7SQ6PM8Q5V387EJCTO', 'HITTypeId': '331OPTJB1UG00H9KAA9T3OG76BQTXE', 'HITGroupId': '3SBA6YTBZ5NVPVR4XA06AJDNWQKOB7', 'CreationTime': datetime.datetime(2022, 5, 11, 20, 32, 13, tzinfo=tzlocal()), 'Title': 'Detect deepfakes appearing for fractions of a second!', 'Description': 'You will be shown short videos of faces, sometimes for fractions of a second. Some of them are deepfakes. Your task is to determine whether the videos shown are real or deepfakes.', 'Question': '<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07

# HIT monitoring helpers

Helper functions that will be useful for monitoring the status of your HIT. See next section for how to use them.

In [6]:
# Contacts MTurk API to get all assignments for a HIT
# Returns them in a list. 
def get_all_assignments(hitid): 
    assignments = []
    should_continue = True
    next_token = False
    while (should_continue): 
        args = {'HITId': hitid, 
                'MaxResults': 100}
        if (next_token): 
            args['NextToken'] = next_token
        r = cl.list_assignments_for_hit(**args)
        next_token = r.get('NextToken', False)
        assignments.extend(r["Assignments"])
        should_continue = len(r["Assignments"]) > 0
    return assignments


def get_hits(max_results=200):
    hits = []
    mr = min(max_results, 100)
    should_continue = True
    next_token=False
    c=0
    while(should_continue):
        args = {
            'MaxResults': mr
        }
        if (next_token): 
            args['NextToken'] = next_token
        r = cl.list_hits(**args)
        next_token = r.get('NextToken', False)
        hits.extend(r["HITs"])
        c += len(r["HITs"])
        should_continue = next_token and (c<max_results)
#         mr = mr-100
    return hits

# Summarizes all hits in `hits` in a human-readable way. 
# Prints out the HIT Title, id, if it is expired, and how many assignments it has
# completed, pending, and left for work. 
def summarize_hits(hits, get_submitted=True): 
    print(len(hits))
    ret = ""
    for hit in hits: 
        expiration = hit['Expiration'].replace(tzinfo=None)
        is_expired = expiration < datetime.datetime.now()
        description = ("Title: {title}\n" 
        "ID: {hid}\n"
        "\tAssignments left: {left}\n"
        "\tAssignments completed: {complete}\n"
        "\tAssignments pending: {pending}\n"
        ).format(
            title=hit['Title'], 
            hid=hit['HITId'], 
            left=hit['NumberOfAssignmentsAvailable'], 
            complete=hit['NumberOfAssignmentsCompleted'], 
            pending=hit['NumberOfAssignmentsPending']
            
        )
        
        if get_submitted:
            submitted=0
            assignments = get_all_assignments(hit['HITId'])
            if assignments:
                for a in assignments: 
                    if a['AssignmentStatus'] == 'Submitted':
                        submitted+=1
                    
            description+='\tAssignments submitted: %d\n' % submitted
        
        description += "\tExpired: {exp}\n\n".format(exp=str(is_expired))
        
        ret += description
    print(ret)
    
# Prints a human-readable summary of all pending/submitted/approved assignments for all hits in `hits`
def summarize_assignments(hits):
    ret = ""
    for hit in hits: 
        hid = hit['HITId']
        title =  hit['Title']
        name = "HIT %s: %s" % (hid, title)
        ret += name + "\n"
        assignments = get_all_assignments(hid)
        if len(assignments) == 0: 
            ret += "\tNo pending/submitted/approved assignments for this HIT\n"
        for a in assignments: 
            desc = "\tAssignment {aid}\n\t\tStatus: {status}\n".format(aid=a['AssignmentId'], status=a['AssignmentStatus'])
            ret += desc
    print(ret)
    
# Refreshes data about the requested hits
def refresh_hits(): 
    global hits 
    global MAX_RESULTS
    hits = cl.list_hits(MaxResults=MAX_RESULTS)['HITs']

# HIT monitoring

In [7]:
# API call to grab HIT data from MTurk 
hits = get_hits(max_results=100)
print(len(hits))


55


In [8]:
# Summarizes all outstanding HITs
# refresh_hits() 
summarize_hits(hits[:16]) 

16
Title: Detect deepfakes appearing for fractions of a second!
ID: 3D3B8GE8946TH2QWWLPFYV13CQVP9F
	Assignments left: 4
	Assignments completed: 4
	Assignments pending: 2
	Assignments submitted: 0
	Expired: False

Title: Detect deepfakes appearing for fractions of a second!
ID: 3DZKABX2ZKK8KKUVUH856DRT4ZZCV4
	Assignments left: 1
	Assignments completed: 3
	Assignments pending: 2
	Assignments submitted: 4
	Expired: False

Title: Detect deepfakes appearing for fractions of a second!
ID: 3ZURAPD28A2Z1KY7E5EH7V5GC8KF1T
	Assignments left: 0
	Assignments completed: 6
	Assignments pending: 3
	Assignments submitted: 1
	Expired: False

Title: Detect deepfakes appearing for fractions of a second!
ID: 3VO4XFFP171A3V3IH16KK407MWT7QC
	Assignments left: 2
	Assignments completed: 5
	Assignments pending: 2
	Assignments submitted: 1
	Expired: False

Title: Detect deepfakes appearing for fractions of a second!
ID: 32FESTC2NJ58755VPRY0ARPAQSQCUF
	Assignments left: 4
	Assignments completed: 0
	Assignments p

In [10]:
# Summarizes assignments for all oustanding HITs 
# refresh_hits()
summarize_assignments(hits)

HIT 3D3B8GE8946TH2QWWLPFYV13CQVP9F: Detect deepfakes appearing for fractions of a second!
	Assignment 3N1FSUEFL8FLIJFO0J7U5D5PD7W4D9
		Status: Submitted
	Assignment 3GNCZX450L2995YVET8GIA8EJ02PAQ
		Status: Submitted
	Assignment 33FOTY3KEP0ULSCRD5THN9G4GNKC1E
		Status: Submitted
	Assignment 3L2IS5HSFDXTLT0G4JCFEMFDQO3UNL
		Status: Submitted
HIT 3DZKABX2ZKK8KKUVUH856DRT4ZZCV4: Detect deepfakes appearing for fractions of a second!
	Assignment 3AAJC4I4FJ7EO235V8JV27DBJMXZJV
		Status: Submitted
	Assignment 3AMYWKA6YE1Q13AR9GEPD512DCLO6N
		Status: Submitted
	Assignment 36PW28KO42B5CE955FS5OY9A5T8EA2
		Status: Submitted
HIT 3ZURAPD28A2Z1KY7E5EH7V5GC8KF1T: Detect deepfakes appearing for fractions of a second!
	Assignment 3DOCMVPBTQTGQ3G77I0T0RNJIRHNNR
		Status: Submitted
	Assignment 31N2WW6R9U5XY7AGRBX1J8G4BKG3FY
		Status: Submitted
	Assignment 34MAJL3QP72N0RDR7U1IS4JDMUA43E
		Status: Submitted
	Assignment 36TFCYNS47PTS1806QSPT9HLYU1XHX
		Status: Submitted
	Assignment 3HRMW88U1957NP30UBJVC0265

# Approve HITs

Approves all outstanding assignments for the HITs displayed above. 

In [9]:
def approve_all(hits): 
    num_approved = 0
    for hit in hits: 
        # make sure you keep getting assignments 
        assignments = get_all_assignments(hit["HITId"])
        #print(assignments)
        for a in assignments: 
            if a['AssignmentStatus'] != 'Approved':
                print("Approving assignment")
                num_approved += 1
                cl.approve_assignment(AssignmentId=a['AssignmentId'])
    print("Approved %d assignments" % num_approved)

In [10]:
# refresh_hits()
approve_all(hits)

Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment
Approving assignment


KeyboardInterrupt: 

# Update expiration or num tasks

In [16]:
# changes the expiration date on a HIT to days_from_now days in the future
def update_expiration(hitid, days_from_now): 
    if ALLOW_UPDATE_EXPIRATION: 
        days = days_from_now*datetime.timedelta(days=1)
        expire_time = datetime.datetime.now() + days

        response = cl.update_expiration_for_hit(HITId=hitid, ExpireAt=expire_time)
        print(response)
        return response
    else: 
        raise RuntimeError("This action is not currently enabled; set `ALLOW_UPDATE_EXPIRATION` to true to proceed with this action")
    
def expire_hits(hits):
    resp = []
    for hit in hits: 
        resp.append(update_expiration(hit, -10))
    return resp

In [17]:
def add_assignments(hitid, num_assignments): 
    if ALLOW_ASSIGNMENT_ADDITION: 
        response = cl.create_additional_assignments_for_hit(
            HITId=hitid,
            NumberOfAdditionalAssignments=num_assignments
        )
        print(response)
        return response
    else: 
        raise RuntimeError("This action is not currently enabled; set `ALLOW_ASSIGNMENT_ADDITION` to true to proceed with this action")

In [14]:
hits

[{'HITId': '3D3B8GE8946TH2QWWLPFYV13CQVP9F',
  'HITTypeId': '331OPTJB1UG00H9KAA9T3OG76BQTXE',
  'HITGroupId': '3SBA6YTBZ5NVPVR4XA06AJDNWQKOB7',
  'CreationTime': datetime.datetime(2022, 5, 11, 20, 32, 16, tzinfo=tzlocal()),
  'Title': 'Detect deepfakes appearing for fractions of a second!',
  'Description': 'You will be shown short videos of faces, sometimes for fractions of a second. Some of them are deepfakes. Your task is to determine whether the videos shown are real or deepfakes.',
  'Question': '<ExternalQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2006-07-14/ExternalQuestion.xsd">\n<ExternalURL>https://cfosco.github.io/timed_df_detection_experiment/?data=json_35.json</ExternalURL>\n  <FrameHeight>700</FrameHeight>\n</ExternalQuestion>',
  'Keywords': 'deepfakes, videos, artifacts, detection, annotate',
  'HITStatus': 'Assignable',
  'MaxAssignments': 10,
  'Reward': '1.00',
  'AutoApprovalDelayInSeconds': 604800,
  'Expiration': datetime.datet

In [19]:
# Use this cell to expire a HIT 
HIT_ids_to_expire = [h['HITId'] for h in hits[:16]] 
expire_hits(HIT_ids_to_expire)

{'ResponseMetadata': {'RequestId': '627905db-d84c-472e-afef-db214101bd06', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '627905db-d84c-472e-afef-db214101bd06', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Thu, 12 May 2022 18:23:32 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '095992bc-c4d9-42cd-86a0-1c6913d4f2b2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '095992bc-c4d9-42cd-86a0-1c6913d4f2b2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Thu, 12 May 2022 18:23:32 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '7ecbfee0-d800-4c0e-884f-3c4d0b7f29a9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7ecbfee0-d800-4c0e-884f-3c4d0b7f29a9', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Thu, 12 May 2022 18:23:32 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '351fbf0f-c07b-4580-b6d7-2d9b90d65390', 'HTTPSt

[{'ResponseMetadata': {'RequestId': '627905db-d84c-472e-afef-db214101bd06',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amzn-requestid': '627905db-d84c-472e-afef-db214101bd06',
    'content-type': 'application/x-amz-json-1.1',
    'content-length': '2',
    'date': 'Thu, 12 May 2022 18:23:32 GMT'},
   'RetryAttempts': 0}},
 {'ResponseMetadata': {'RequestId': '095992bc-c4d9-42cd-86a0-1c6913d4f2b2',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amzn-requestid': '095992bc-c4d9-42cd-86a0-1c6913d4f2b2',
    'content-type': 'application/x-amz-json-1.1',
    'content-length': '2',
    'date': 'Thu, 12 May 2022 18:23:32 GMT'},
   'RetryAttempts': 0}},
 {'ResponseMetadata': {'RequestId': '7ecbfee0-d800-4c0e-884f-3c4d0b7f29a9',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amzn-requestid': '7ecbfee0-d800-4c0e-884f-3c4d0b7f29a9',
    'content-type': 'application/x-amz-json-1.1',
    'content-length': '2',
    'date': 'Thu, 12 May 2022 18:23:32 GMT'},
   'RetryAttempts': 0}},
 {'ResponseMe

In [ ]:
# Use this cell to add assignments to a HIT 
HIT_id_to_add_assignments = "FILL THIS IN"
num_assignments_to_add = 0
add_assignments(HIT_id_to_add_assignments, num_assignments_to_add)

# Add custom qualifications 

## Add a qualification to disqualify workers who have done work before

- uses "negative qualification" method from https://github.com/cloudyr/MturkR/wiki/qualifications-as-blocks

#### NOTE: quals are kept separate for the sandbox and prod. Make sure you are creating and assigning your quals in prod. 

### Structure of a new qualification

In [61]:
NEW_QUAL = {
    'Name': 'qualName',
    'Keywords': 'Keywords for qual',
    'Description': 'What is this qual, and why are you assigning it?',
    'QualificationTypeStatus': 'Active',
    'AutoGranted': False
}

### Helpers for creating, viewing, and assigning qualifications

In [112]:
# Registers a custom qualification with MTurk 
def create_qual(new_qual):
    if ALLOW_CREATE_QUAL: 
        response = cl.create_qualification_type(**new_qual)
        print(response)
        Id = response['QualificationType']['QualificationTypeId']
        print("id", Id)
        return Id
    else: 
        raise RuntimeError("This action is not currently enabled; set `ALLOW_CREATE_QUAL` to true to proceed with this action")
        
# Gets all the custom quals you have created and prints them
def list_quals(): 
    response = cl.list_qualification_types(
            Query='didntPassVigilanceTimedDFExp',
            MustBeRequestable=False
    )
    print(response)
    
# Assigns a qualification to a worker 
def assign_qual(qual_id, worker_ids): 
    for worker in worker_ids: 
        response = cl.associate_qualification_with_worker(
                QualificationTypeId=qual_id, 
                WorkerId=worker,
                IntegerValue=1,
                SendNotification=False
        )
        print(response)
        assert response
        
def remove_qual(qual_id, worker_ids):
    for worker in worker_ids:
        response = cl.disassociate_qualification_from_worker(
                QualificationTypeId=qual_id,
                WorkerId=worker)
        print(response)
        assert response

# Gets the ids of all workers who worked on a particular hit 
def get_workers_for_hit(hitid): 
    a = get_all_assignments(hitid)
    workers = [a_['WorkerId'] for a_ in a]
    return workers
    
# Confirms that every worker in worker_ids has qual with qual_id
def confirm_quals(qual_id, worker_ids): 
    for w in worker_ids: 
        response = cl.get_qualification_score(
                QualificationTypeId=qual_id,
                WorkerId=w
        )
        response = response['Qualification']
        assert response['Status'] == 'Granted'
        assert response['IntegerValue'] == 1
        
# Assigns qual with `qual_id` to every worker who has completed an assignment for the hit with `hitid`
def assign_qual_for_hit(hitid, qual_id): 
    workers = get_workers_for_hit(hitid)
    print("got workers")
    assign_qual(qual_id, workers)
    print("assigned qual")
    confirm_quals(qual_id, workers)
    print("confirmed qual")

### Use the following cells to manipulate qualifications

In [74]:
# Use this cell to view the custom qualifications you have created
list_quals()

{'NumResults': 1, 'NextToken': 'p2:wTmsdOwRpjinNTJmffbOgvjbk/ggo7/20sTn4WB4DZnS8Jdhjw2SLWxKNCiTRg==', 'QualificationTypes': [{'QualificationTypeId': '3CNIZ8EIUXRXLUMEOJNMO8EPWSI3BW', 'CreationTime': datetime.datetime(2022, 5, 9, 22, 10, 43, tzinfo=tzlocal()), 'Name': 'didntPassVigilanceTimedDFExp', 'Description': 'Assigned to turkers that played our experiment and did not correctly pass the attention checks', 'Keywords': 'vigilance deepfake experiment', 'QualificationTypeStatus': 'Active', 'IsRequestable': True, 'AutoGranted': False}], 'ResponseMetadata': {'RequestId': '4c4b8f1b-d7aa-4270-8fb1-1ff61d8d7821', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4c4b8f1b-d7aa-4270-8fb1-1ff61d8d7821', 'content-type': 'application/x-amz-json-1.1', 'content-length': '475', 'date': 'Tue, 10 May 2022 07:11:47 GMT'}, 'RetryAttempts': 0}}


In [ ]:
# Use this cell to assign a custom qual to every worker who has done a specific HIT
# hit_id = "FILL THIS IN"
# qual_id_to_assign = "FILL THIS IN"
# assign_qual_for_hit(hit_id, qual_id_to_assign)

# Blacklist qualifications

In [97]:
# Use this cell to create blacklist qual 
blacklist_qual = {
    'Name': 'didntPassVigilanceTimedDFExp',
    'Keywords': 'vigilance deepfake experiment',
    'Description': 'Assigned to turkers that played our experiment and did not correctly pass the attention checks',
    'QualificationTypeStatus': 'Active',
    'AutoGranted': False
}

blacklist_qual_id = create_qual(blacklist_qual)

{'QualificationType': {'QualificationTypeId': '376OKSJ89UCMZGQ2VRG1LX76M8D7XO', 'CreationTime': datetime.datetime(2022, 5, 10, 5, 19, 19, tzinfo=tzlocal()), 'Name': 'didntPassVigilanceTimedDFExp', 'Description': 'Assigned to turkers that played our experiment and did not correctly pass the attention checks', 'Keywords': 'vigilance deepfake experiment', 'QualificationTypeStatus': 'Active', 'IsRequestable': True, 'AutoGranted': False}, 'ResponseMetadata': {'RequestId': 'f1067be4-77a3-4c91-a906-f595e546ba02', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f1067be4-77a3-4c91-a906-f595e546ba02', 'content-type': 'application/x-amz-json-1.1', 'content-length': '375', 'date': 'Tue, 10 May 2022 09:19:18 GMT'}, 'RetryAttempts': 0}}
id 376OKSJ89UCMZGQ2VRG1LX76M8D7XO


In [76]:
# Define blacklist
blacklist_worker_IDs = ['A3LT6J8SHD7XQS',
                        'A32B8FMRQ04EMF',
                        'A25IIR03W5YGLG',
                        'A2DBHKZHJL03AJ',
                        'AHEMKTCJLHDKF',
                        'A1BU9TYDZ71T1G',
                        'A14KI8Q5500WNH',
                        'ALQ6D6V0WWGE5',
                        'A36CB3KV9UVE1T',
                        'A7QBBXOG4ERRC',
                        'AA6GNJJ8US627',
                        'ALF1RTBFSYHNE',
                        'A3P51HLH8H41ZS',
                        'A2OI65EUOJ0CBY',
                        'A2KFJ8AKBP6QQF',
                        'A258TMYFKRH6PB',
                        'A3AFM919BJLBU3',
                        'A22O4X7C3A76J6',
                        'A2DV7L9U70HKSV',
                        'A1TVWMWE03AWAW',
                        'A1IFDTZI886C7M',
                        'A190X9GHHXNC2P',
                        'A31P89TWI7EOWB',
                        'A2HW1TJJP1J6RT',
                        'A1IFDTZI886C7M',
                        ]

In [77]:
assign_qual('3CNIZ8EIUXRXLUMEOJNMO8EPWSI3BW', blacklist_worker_IDs)

{'ResponseMetadata': {'RequestId': 'ae4b5815-3cee-4ce7-9d72-b9024fb47ed2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ae4b5815-3cee-4ce7-9d72-b9024fb47ed2', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Tue, 10 May 2022 08:28:34 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': 'c6011a76-5125-4ade-b4bf-6d1edae12b34', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c6011a76-5125-4ade-b4bf-6d1edae12b34', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Tue, 10 May 2022 08:28:34 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '6554fb0d-82b7-41be-89ab-2d3c8588a434', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6554fb0d-82b7-41be-89ab-2d3c8588a434', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Tue, 10 May 2022 08:28:34 GMT'}, 'RetryAttempts': 0}}
{'ResponseMetadata': {'RequestId': '3bc479be-b338-4bea-89dd-0df290980469', 'HTTPSt

In [113]:
# FOR SANDBOX TESTING
remove_qual('376OKSJ89UCMZGQ2VRG1LX76M8D7XO', ['ACUP2HVXGZY46'])

{'ResponseMetadata': {'RequestId': '83efe4d8-82b7-48f7-9fcc-5053955c293e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '83efe4d8-82b7-48f7-9fcc-5053955c293e', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Tue, 10 May 2022 09:23:17 GMT'}, 'RetryAttempts': 0}}


# Create Compensation HIT

Mistakes happen, and sometimes they can lead to a worker who put in an honest effort being unable to complete a task and get paid. It's a good idea to compensate these workers when they reach out because it helps maintain relations with workers and is the right thing to do.

However, workers can only be paid upon completing a task. The workaround is to create a custom qualification, assign it to the worker you want to compensate, and create a no-work HIT requiring the custom qualification. This code does that.

In [13]:
# worker_ids is str[]
# compensation is str but should match the regex ^\d*\.\d\d$ (e.g. "1.00")
# for_hit_id is str -- optional, but helpful for records
def compensate_workers(worker_ids, compensation, for_hit_id=""):
    with open('compensation.xml', 'r') as myfile:
        question_xml=myfile.read()

    keywords = 'compensation'
    description = 'Compensation for HIT'
    if for_hit_id:
        keywords += ', ' + for_hit_id
        description += ' ' + for_hit_id

    # create qual, assign to workers
    custom_qual = {
        'Name': str(uuid4()), # a qual must have a unique name
        'Keywords': keywords,
        'Description': description,
        'QualificationTypeStatus': 'Active',
        'AutoGranted': False
    }
    qual_id = create_qual(custom_qual)
    assign_qual(qual_id, worker_ids)

    # create HIT requiring qual
    task = {
        'numAssignments': len(worker_ids),
        'lifetime': 3 * 24 * 60 * 60, # 3 days
        'duration': 5 * 60, # 5 min
        'rewardAmount': compensation,
        'title': description,
        'keywords': keywords,
        'description': description,
    }
    quals = [{
        'QualificationTypeId': qual_id,
        'Comparator': 'Exists',
        'ActionsGuarded': 'DiscoverPreviewAndAccept'
    }]
    create_hit(task, question_xml, quals)

In [44]:
worker_ids = ['A10V73YYJVB362'] # worker_id strings in a list
compensation = "1.5" # change to the amount of dollars you want to give
for_hit_id = "3Y6OYEUGEAT8Y72O3HQ2R2MXROYS5N" # hit_id string (what you are compensating for)
compensate_workers(worker_ids, compensation, for_hit_id)

{'QualificationType': {'QualificationTypeId': '3X9YVAW8NXRJWT9CUA5YXGPLXLK4KX', 'CreationTime': datetime.datetime(2020, 5, 24, 18, 17, 29, tzinfo=tzlocal()), 'Name': 'c663c77c-744b-48b4-b64e-a5b84cc77010', 'Description': 'Compensation for HIT 3Y6OYEUGEAT8Y72O3HQ2R2MXROYS5N', 'Keywords': 'compensation, 3Y6OYEUGEAT8Y72O3HQ2R2MXROYS5N', 'QualificationTypeStatus': 'Active', 'IsRequestable': True, 'AutoGranted': False}, 'ResponseMetadata': {'RequestId': 'cd7ff091-c60f-4e1d-ac93-ff5f00c1b924', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cd7ff091-c60f-4e1d-ac93-ff5f00c1b924', 'content-type': 'application/x-amz-json-1.1', 'content-length': '355', 'date': 'Sun, 24 May 2020 22:17:28 GMT'}, 'RetryAttempts': 0}}
id 3X9YVAW8NXRJWT9CUA5YXGPLXLK4KX
{'ResponseMetadata': {'RequestId': '1bbd3abc-b3e9-4cf5-8d66-e3963adcdabe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '1bbd3abc-b3e9-4cf5-8d66-e3963adcdabe', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2

# Download data

Helper to download data from MTurk 

In [16]:
def pretty_print(obj):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(obj)
    pp = None

# Downloads all the assignments completed for `hits` as a list of dictionaries. 
# If a download_path is given, also saves that data as json 
def get_assignment_content(hits, download_path="", should_print=False): 
    all_responses = []
    for hit in hits: 
        hitid = hit['HITId']
        assignments = get_all_assignments(hitid)
        for a in assignments:
            a_xml = a['Answer']
            #print(a_xml)
            soup = bs(a_xml, "html.parser")
            answers = soup.find_all("answer")
            #print(answers)
            results = {'HITId': a['HITId'], 'AssignmentId': a['AssignmentId'], 'WorkerId': a['WorkerId']}
            for ans in answers: 
                identifier = ans.find('questionidentifier').string
                answer = ans.find('freetext').string
                try: 
                    results[identifier] = json.loads(answer)
                except:
                    results[identifier] = answer
            all_responses.append(results)
    if should_print: 
        pretty_print(all_responses)
    if download_path: 
        with open(download_path, 'w') as outfile: 
            json.dump(all_responses, outfile)
    return all_responses
            

In [17]:
# Define name of file to save to
download_path = './53assig_may12.json'

In [18]:
# Use this cell to download data
responses = get_assignment_content(hits[:16], download_path=download_path, should_print=False)
print('Individual assignments downloaded:',len(responses))

Individual assignments downloaded: 53


In [19]:
# View assignments
import collections

def response_counts(json_file):
    # Load json file
    with open(json_file, 'r') as infile:
        data = json.load(infile)
    
    # Loop over all HITs and count responses
    res = []
    for hit in data:
        for r in hit["results"]:
            res.append(r["response"])

    # Show counts in res
    counts = collections.Counter(res)
    print(counts)

response_counts(download_path)

Counter({'REAL': 3084, 'FAKE': 2746})


In [20]:
# Show counts of VIG, VIG_TEXT, CARI and REAL for each level
def response_counts_by_level(json_file):
    # Load json file
    with open(json_file, 'r') as infile:
        data = json.load(infile)

    # Loop over HITs and get number of VIG, VIG_TEXT, CARI and REAL in each level
    for i,h in enumerate(data):
        res = []
        for r in h['results']:
            # Count VIG, VIG_TEXT, CARI and REAL
            res.append(r['label'])
        
        # Show counts in res
        counts = collections.Counter(res)

        print("HIT %d:"%i, counts)

response_counts_by_level(download_path)


HIT 0: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 1: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 2: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 3: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 4: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 5: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 6: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 7: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 8: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 9: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 10: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 11: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5, 'VIG_TEXT': 5})
HIT 12: Counter({'REAL': 50, 'CARI': 25, 'FAKE': 25, 'VIG': 5,

In [26]:

def compare_two_hits(json_file, h1, h2):
    '''Loop over all responses in a HIT and check for differences in labels'''
    
    # Load json file
    data = json.load(open(json_file, 'r'))

    # Get results for h1 and h2
    h1_results = data[h1]['results']
    h2_results = data[h2]['results']

    print("Comparing HIT %d (%d trials) to HIT %d (%d trials)" % (h1, len(h1_results), h2, len(h2_results)))

    for i in range(len(h1_results)):
        if h1_results[i]['label'] != h2_results[i]['label'] or h1_results[i]['video'] != h2_results[i]['video']:
            print("\n\nWARNING: Difference in trial %d (assuming trial %d of level %d): %s != %s" % (i, i%22, i//22, h1_results[i]['label'], h2_results[i]['label']))
            print("Full response for HIT %d:" % h1)
            print(h1_results[i])
            print("Full response for HIT %d:" % h2)
            print(h2_results[i])


compare_two_hits(download_path, 0, 3)

Comparing HIT 0 (110 trials) to HIT 3 (110 trials)


In [19]:
def compare_HIT_with_ground_truth(json_file, hit_id, ground_truth_file):

    # Load json file
    data = json.load(open(json_file, 'r'))

    # Get results for hit
    hit_results = data[hit_id]['results']

    # load ground truth
    ground_truth = json.load(open(ground_truth_file, 'r'))

    j=0
    for lvl, trial in ground_truth.items():
        for i in range(len(trial)):
            if trial[i]['url'] != hit_results[j]['video']:
                print("\n\nWARNING: Difference in %s, trial %d): %s != %s" % (lvl, i, trial[i]['url'], hit_results[j]['video']))
            j+=1


for i in range(10):
    compare_HIT_with_ground_truth(download_path, i, '../src/jsons_2vig/json_22.json')
